In [1]:
%%capture
%pip install --upgrade iblm

In [2]:
%pip list | grep iblm

iblm                         0.3.54

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
sys.path.append("../..")
import pandas as pd
import numpy as np

import iblm
#from src.iblm.iblbagging.iblbagging import IBLBaggingModel
from sklearn.datasets import make_classification

In [10]:
num_train_data = 10000 # 50〜300
num_test_data = 1000
sample = num_train_data + num_test_data
n_informative = 2
n_redundant = 0
n_features = n_informative + n_redundant
weights = [0.5, 0.5]
flip_y=0
seed = 3655  # 3655,3656,3657

# testデータの個数を揃えるためにtrain_test_splitを使っていない
x, y = make_classification(
    n_samples = sample,  # データ数
    n_features = n_features,  # 特徴量の数
    n_informative = n_informative,  # ラベル予測に意味のある特徴量の数
    n_redundant = n_redundant,  # 冗長な特徴量
    weights = weights,  # [0,1]の割合
    flip_y = flip_y, # 逆のラベルに反転する割合
    random_state = seed
    )

x = np.round(x, decimals=3)
x = pd.DataFrame(x)
y = y.astype(int)

x_test = x[0:num_test_data]
x_train = x[num_test_data:]

y_test = y[:num_test_data]
y_train = y[num_test_data:]

In [11]:
iblbg = iblm.IBLBaggingModel(
    api_type="openai",
    #api_type="gemini",
    model_name="gpt-4-0125-preview",
    #model_name="gemini-pro",
    objective="binary",
    num_model=20,
    max_sample = 2000,
    min_sample = 300,
    )

In [12]:
code_models = iblbg.fit(
    x_train,
    y_train,
    temperature=0.3
    )

model_1


2024-02-16 03:29:57,345 [iblm.ibl][INFO] (ibl:ibl.py:fit:150)


#########
def predict(x):
    import numpy as np

    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Custom simple logistic regression-like model
        # Coefficients are manually adjusted based on the dataset characteristics
        # Intercept
        b0 = 0.5
        # Coefficients for the features
        b1, b2 = 0.1, 0.15
        
        # Linear combination of input features
        linear_combination = b0 + b1*row[0] + b2*row[1]
        # Sigmoid function to map the linear combination to (0,1)
        y = 1 / (1 + np.exp(-linear_combination))
        
        output.append(y)
    return np.array(output)
#########
model_2


In [ ]:
code_models

In [ ]:
top_model = 20

for i in range(1, top_model):
    y_pred = iblbg.predict_(x_test, i)
    print(iblbg.evaluate(y_test, y_pred))

{'roc_auc': 0.9049601339148055, 'pr_auc': 0.8784086458731357, 'accuracy': 0.811, 'recall': 0.9439071566731141, 'precision': 0.7530864197530864, 'f1_score': 0.8377682403433476}
{'roc_auc': 0.9049601339148055, 'pr_auc': 0.8784086458731357, 'accuracy': 0.811, 'recall': 0.9439071566731141, 'precision': 0.7530864197530864, 'f1_score': 0.8377682403433476}
{'roc_auc': 0.9049601339148055, 'pr_auc': 0.878409322056068, 'accuracy': 0.811, 'recall': 0.9439071566731141, 'precision': 0.7530864197530864, 'f1_score': 0.8377682403433476}
{'roc_auc': 0.903412344670439, 'pr_auc': 0.875054736147716, 'accuracy': 0.791, 'recall': 0.9497098646034816, 'precision': 0.728486646884273, 'f1_score': 0.8245172124265323}
{'roc_auc': 0.900949497619248, 'pr_auc': 0.8714041633628898, 'accuracy': 0.825, 'recall': 0.9323017408123792, 'precision': 0.77491961414791, 'f1_score': 0.8463564530289728}
{'roc_auc': 0.8959397063004835, 'pr_auc': 0.8657062797864831, 'accuracy': 0.825, 'recall': 0.9226305609284333, 'precision': 0.7